In [1]:
from faker import Faker
import sqlite3
import random
from datetime import timedelta, datetime

In [2]:
conn = sqlite3.connect('health_tracker.db')
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON")

In [3]:
fake = Faker()
fake.unique.name()
Faker.seed(424243)
random.seed(424243)

In [4]:
def insert_sample_user_data(num_records):
    for _ in range(num_records):
        name = fake.name()
        dob = fake.date_of_birth(minimum_age=18, maximum_age=50)
        reg_date = fake.date_this_year()
        gender = random.choices(['Male', 'Female', 'PreferNotToSay'], [0.45, 0.45, 0.1], k=1)[0]
        email = fake.unique.email()
        uname = fake.unique.user_name()

        dob_str = dob.strftime('%Y-%m-%d')
        reg_date_str = reg_date.strftime('%Y-%m-%d')

        say_info = random.choices([True, False], [0.7,0.3], k=1)[0]
        if say_info:
            first_name = name.split()[0]
            last_name = name.split()[1]
            cursor.execute('''
            INSERT INTO users (
            first_name,
            last_name,
            date_of_birth,
            registration_date,
            gender,
            username,
            email
            )
            VALUES (?, ?, ?, ?, ?, ?, ?);
            ''', (first_name, last_name, dob_str, reg_date_str, gender, uname, email))
        else:
            cursor.execute('''
            INSERT INTO users (
            first_name,
            last_name,
            date_of_birth,
            registration_date,
            gender,
            username,
            email
            )
            VALUES (?, ?, ?, ?, ?, ?, ?);
            ''', (None, None, None, reg_date_str, gender, uname, email))


In [5]:
def insert_sample_device_data():
    cursor.execute("SELECT COUNT(*) FROM users")
    number_of_rows = cursor.fetchone()[0]
    cursor.execute("SELECT user_id FROM users")
    user_ids = [row[0] for row in cursor.fetchall()]

    for _ in range(number_of_rows-number_of_rows//10):
        user_id = random.choice(user_ids)
        dev_type = random.choice(['watch', 'band'])
        if dev_type=='watch':
            device_model = random.choice(['watch 3 ultimate', 'watch 3', 'watch 2', 'watch 1'])
        else:
            device_model = random.choice(['band 4', 'band 5', 'band 3', 'band 2', 'band 1'])
        firmware_version = random.choice(['1.1.2', '1.1.1a', '1.2.1b', '1.0.5', '1.1.3'])
        warranty_expire_date = fake.future_date(end_date="+2y").strftime('%Y-%m-%d')
        last_sync_date = fake.date_between(start_date='-30d').strftime('%Y-%m-%d')
        cursor.execute('''
                        INSERT INTO devices (
                            user_id,
                            device_type,
                            device_model,
                            firmware_version,
                            warranty_expire_date,
                            last_sync_date
                        )
                        VALUES (?, ?, ?, ?, ?, ?);
                        ''', (user_id, dev_type, device_model, firmware_version, warranty_expire_date, last_sync_date))

In [6]:
def insert_sample_exercise_data():
        exercises = ['push-ups', 'lifting', 'flexibility', 'running', 'bodyweight exercises']
        units =     ['repetitions', 'kg', '%', 'km', 'sets']
        for i in range(len(exercises)):
            cursor.execute('''
                            INSERT INTO exercise_type (
                                exercise_name,
                                exercise_unit
                            )
                            VALUES (?,?);
                            ''', (exercises[i], units[i]))

In [7]:
def insert_sample_metric_data():
        metric_name = ['Heart Rate', 'Steps', 'sleep', 'SpO2', 'Blood Pressure']
        metric_unit = ['bpm', 'steps', 'time', '%', 'mmHg']
        for i in range(len(metric_name)):
            cursor.execute('''
                            INSERT INTO metric_type (
                                metric_name,
                                metric_unit
                            )
                            VALUES (?,?);
                            ''', (metric_name[i], metric_unit[i]))

In [8]:
def insert_sample_exerciseslog_data():
    cursor.execute("SELECT COUNT(*) FROM users")
    number_of_rows = cursor.fetchone()[0]
    cursor.execute("SELECT device_id FROM devices")
    device_ids = [row[0] for row in cursor.fetchall()]
    cursor.execute("SELECT exercise_id FROM exercise_type")
    exercises = [row[0] for row in cursor.fetchall()]
    
    for _ in range(number_of_rows*2):
        device_id = random.choice(device_ids)
        exercise_id = random.choice(exercises)
        if exercise_id==1:
            exercise_value = int(random.normalvariate(40, 10))
        elif exercise_id==2:
            exercise_value = int(random.normalvariate(70, 15))
        elif exercise_id==3:
            exercise_value = random.uniform(0.2, 1.0)
        elif exercise_id==4:
            exercise_value = int(random.normalvariate(5, 0.5))
        elif exercise_id==5:
            exercise_value = int(random.normalvariate(10, 1))
        
        now = datetime.now()

        one_week_ago = now - timedelta(weeks=1)

        start_time = one_week_ago + (now - one_week_ago) * random.random()
        end_time = start_time + timedelta(minutes=1)*random.random() + timedelta(hours=3)*random.random()
        cursor.execute('''
                        INSERT INTO exercises_log (
                            exercise_id,
                            device_id,
                            exercise_value,
                            start_time,
                            end_time
                        )
                        VALUES (?, ?, ?, ?, ?);
                        ''', (exercise_id, device_id, exercise_value, start_time, end_time))

In [9]:
def insert_sample_metricslog_data():
    cursor.execute("SELECT device_id FROM devices")
    device_ids = [row[0] for row in cursor.fetchall()]
    
    for i in device_ids:
        for time in range(12*24*7):
            device_id = i
            
            metric_value_1 = int(random.normalvariate(70, 10))

            stepped = random.randint(1, 10)
            if stepped < 3:
                metric_value_2 = int(random.normalvariate(60, 60))
                metric_value_2 = max(metric_value_2, 0)
            else:
                metric_value_2 = int(random.normalvariate(5, 1))

            metric_value_3 = random.normalvariate(5, 1)
            now = datetime.now()

            timestamp = now - timedelta(minutes=5)*time

            if timestamp.hour >= 22 or timestamp.hour <= 8:
                cursor.execute('''
                            INSERT INTO metrics_log (
                                metric_id,
                                device_id,
                                metric_value,
                                metric_timestamp
                            )
                            VALUES (?, ?, ?, ?);
                            ''', (1, device_id, metric_value_1, timestamp))
                cursor.execute('''
                            INSERT INTO metrics_log (
                                metric_id,
                                device_id,
                                metric_value,
                                metric_timestamp
                            )
                            VALUES (?, ?, ?, ?);
                            ''', (3, device_id, metric_value_3, timestamp))
            else:
                cursor.execute('''
                                INSERT INTO metrics_log (
                                    metric_id,
                                    device_id,
                                    metric_value,
                                    metric_timestamp
                                )
                                VALUES (?, ?, ?, ?);
                                ''', (1, device_id, metric_value_1, timestamp))
                cursor.execute('''
                                INSERT INTO metrics_log (
                                    metric_id,
                                    device_id,
                                    metric_value,
                                    metric_timestamp
                                )
                                VALUES (?, ?, ?, ?);
                                ''', (2, device_id, metric_value_2, timestamp))

In [10]:
def insert_sample_goal_data():
    cursor.execute("SELECT COUNT(*) FROM users")
    number_of_rows = cursor.fetchone()[0]
    cursor.execute("SELECT user_id FROM users")
    user_ids = [row[0] for row in cursor.fetchall()]

    for _ in range(number_of_rows*3):
        user_id = random.choice(user_ids)
        target_type = random.choice(['Exercise', 'HealthMetric'])
        if target_type == 'HealthMetric':
            metric_id = random.randint(1,2)
            if metric_id == 1:
                target_value = random.randint(50, 70)
            else:
                target_value = int(random.normalvariate(5000, 250))
            now = datetime.now()

            one_week_ago = now - timedelta(weeks=1)

            start_time = one_week_ago + (now - one_week_ago) * random.random()
            cursor.execute('''
                            INSERT INTO goals (
                                user_id,
                                metric_id,
                                exercise_id,
                                target_type,
                                target_value,
                                goal_start_date
                            )
                            VALUES (?, ?, ?, ?, ?, ?);
                            ''', (user_id, metric_id, None, target_type, target_value, start_time))
        else:
            exercise_id = 1
            target_value = random.randint(50, 70)
            now = datetime.now()

            one_week_ago = now - timedelta(weeks=1)

            start_time = one_week_ago + (now - one_week_ago) * random.random()
            cursor.execute('''
                            INSERT INTO goals (
                                user_id,
                                metric_id,
                                exercise_id,
                                target_type,
                                target_value,
                                goal_start_date
                            )
                            VALUES (?, ?, ?, ?, ?, ?);
                            ''', (user_id, None, exercise_id, target_type, target_value, start_time))


In [11]:
insert_sample_user_data(20)

In [12]:
insert_sample_device_data()

In [13]:
insert_sample_exercise_data()

In [14]:
insert_sample_metric_data()

In [15]:
insert_sample_exerciseslog_data()

/tmp/ipykernel_21339/2799765046.py:29: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''


In [16]:
insert_sample_metricslog_data()

/tmp/ipykernel_21339/3023290985.py:43: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''
/tmp/ipykernel_21339/3023290985.py:52: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''
/tmp/ipykernel_21339/3023290985.py:24: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''
/tmp/ipykernel_21339/3023290985.py:33: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''


In [17]:
insert_sample_goal_data()

/tmp/ipykernel_21339/480264677.py:21: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''
/tmp/ipykernel_21339/480264677.py:40: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''


In [18]:
conn.commit()
cursor.close()
conn.close()